In [1]:
from sklearn.preprocessing import StandardScaler
from utils import enrich_data, prepare_training_data
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt  # Import the matplotlib.pyplot module
import os
import pandas as pd
import shutil  # Import the shutil module for file operations

# Define the path to the data folder
data_folder = 'data/learn'
output_folder = 'training_plots'  # Specify the output folder

# Clean the output folder before each run
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # Remove the folder and its contents

# Create the output folder
os.makedirs(output_folder)


# Get a list of all files in the data folder
file_list = [filename for filename in os.listdir(data_folder) if filename.endswith('.csv')]

# Loop through each file
for filename in file_list:
    file_path = os.path.join(data_folder, filename)
    print(file_path)

    # Prepare Data
    df, labeled_data, df_for_training, cols = enrich_data(file_path)
    #print(df_for_training)

    # LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = StandardScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    #print('df_for_training', df_for_training)

    ## Prepare training data
    trainX, trainY = prepare_training_data(df_for_training_scaled)
    #print('trainY', trainY)

    # Load model weights if available
    try:
        model = load_model('a_model')
        print("Model loaded successfully.")
    except:
        print("No model found. Training from scratch.")
        model = Sequential()
        model.add(LSTM(64, activation='relu', input_shape=(
            trainX.shape[1], trainX.shape[2]), return_sequences=True))
        model.add(LSTM(32, activation='relu', return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(trainY.shape[1]))
        model.compile(optimizer='adam', loss='mse')

    #model.summary()

    # fit the model
    history = model.fit(trainX, trainY, epochs=10, batch_size=10, validation_split=0.1, verbose=1)
    plt.plot(history.history['loss'], label='Training loss')
    plt.plot(history.history['val_loss'], label='Validation loss')
    plt.legend()
    model.save('a_model')

    # Save the plot as an image file in the output folder
    output_filename = os.path.join(output_folder, os.path.splitext(filename)[0] + '_plot.png')
    plt.savefig(output_filename)

    # Clear the current figure
    plt.clf()



data/learn\AAPL.csv
df_for_training              Open        High         Low       Close   HighLow  CloseOpen  \
0        0.113281    0.113281    0.112723    0.112723  0.000558  -0.000558   
1        0.115513    0.116071    0.115513    0.115513  0.000558   0.000000   
2        0.118862    0.119420    0.118862    0.118862  0.000558   0.000000   
3        0.126116    0.126674    0.126116    0.126116  0.000558   0.000000   
4        0.132254    0.132813    0.132254    0.132254  0.000559   0.000000   
...           ...         ...         ...         ...       ...        ...   
10749  182.130005  183.130005  177.350006  178.850006  5.779999  -3.279999   
10750  179.690002  180.270004  177.580002  179.800003  2.690002   0.110001   
10751  180.869995  180.929993  177.009995  178.190002  3.919998  -2.679993   
10752  179.479996  180.750000  177.600006  177.970001  3.149994  -1.509995   
10753  177.320007  178.619995  176.550003  177.789993  2.069992   0.469986   

       TrendLabel  
0      

INFO:tensorflow:Assets written to: a_model\assets


data/learn\META.csv
df_for_training             Open        High         Low       Close    HighLow  CloseOpen  \
0      32.610001   33.590000   30.940001   31.000000   2.649999  -1.610001   
1      31.370001   32.500000   31.360001   32.000000   1.139999   0.629999   
2      32.950001   33.209999   31.770000   33.029999   1.439999   0.079998   
3      32.900002   32.950001   31.110001   31.910000   1.840000  -0.990002   
4      31.480000   31.690001   28.650000   28.840000   3.040001  -2.640000   
...          ...         ...         ...         ...        ...        ...   
2819  313.230011  317.070007  310.459991  316.559998   6.610016   3.329987   
2820  314.399994  317.890015  310.109985  312.640015   7.780030  -1.759979   
2821  312.880005  313.630005  302.850006  305.209991  10.779999  -7.670014   
2822  307.940002  312.339996  303.869995  305.739990   8.470001  -2.200012   
2823  302.570007  304.720001  300.359985  301.640015   4.360016  -0.929992   

      TrendLabel  
0       

INFO:tensorflow:Assets written to: a_model\assets


<Figure size 640x480 with 0 Axes>